In [1]:
import json
import cv2
from shapely.geometry import Polygon
import itertools
import shutil

### Extract json data and transform into required COCO format

In [2]:
# first we get the image size
# since all taken in iPhone, we just need to check 1 image
im = cv2.imread('data/listerine_real/IMG_0181.jpg')
h, w, c = im.shape
print(h, w)

4032 3024


In [3]:
# since we are using listerine, we set the category ID
cat_id = 6

In [4]:
# read data from the json file
with open('data/listerine_real/listerine_json.json') as f:
    d = json.load(f)

In [5]:
# open train and mask json
with open('data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/train/images/annotations.json') as f:
    train = json.load(f)
    
with open('data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/val/images/annotations.json') as f:
    val = json.load(f)

In [6]:
# we start image & mask ID from 1000 to avoid overlap with synthetic data
image_count = 1000
mask_count = 1000

# store the images we filtered into both train and val datasets
train_imgs = []
val_imgs = []

for key in d.keys():
    # this is the dictionary to append to images list
    dict_1 = {
            "file_name": d[key]['filename'],
            "height": h,
            "width": w,
            "id": image_count 
            }
    
    # this logic is used to split real image dataset equally into train n val
    if image_count < 1015:
        train['images'].append(dict_1)
        train_imgs.append(d[key]['filename'])
    else:
        val['images'].append(dict_1)
        val_imgs.append(d[key]['filename'])

        
    for segment in d[key]['regions']:
        x = segment['shape_attributes']['all_points_x']
        y = segment['shape_attributes']['all_points_y']
        
        # get the area of the polygon from the points
        pgon = Polygon(zip(x, y))
        area = pgon.area
        
        # combine the x and y coordinates into 1 alternating list
        coords = [i for i in itertools.chain.from_iterable(itertools.zip_longest(x,y)) if i]
        
        # get bbox coordinates
        bbox = [min(x), min(y), max(x), max(y)]
        
        # this is the dictionary to append to annotations list
        dict_2 = {
                "segmentation": [ coords ],
                "area": area,
                "iscrowd": 0,
                "image_id": image_count,
                "bbox": bbox,
                "category_id": cat_id,
                "id": mask_count
                }
        
        # this logic is used to split real image dataset equally into train n val
        if image_count < 1015:
            train['annotations'].append(dict_2)
        else:
            train['annotations'].append(dict_2)
        
        mask_count += 1
        
    image_count += 1

In [7]:
# save new info back to train and mask json
with open('data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/train/images/annotations.json', 'w') as f:
    json.dump(train, f)
    
with open('data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/val/images/annotations.json', 'w') as f:
    json.dump(val, f)

### Copy images into their respective folders

In [8]:
for file in train_imgs:
    file_path = 'data/listerine_real/' + file
    shutil.copy(file_path, 'data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/train/images/')

for file in val_imgs:
    file_path = 'data/listerine_real/' + file
    shutil.copy(file_path, 'data/listerine#23-11-21#05 20 PM/61be142d-ca8c-4e0e-95b9-752494d232d4/val/images/')